# Phising email detection and reply system powered by LLMs
### Autors: Aleksander Folfas, Hong Jing

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text  
from tensorflow import keras
from tensorflow.keras.models import load_model

In [3]:
import tensorflow as tf
print(tf.__version__)

print(hub.__version__)

2.9.2
0.16.1


## Initialization

In [ ]:
%run ReplyGeneration.ipynb # running this notebook will download Mistral LLM if its not installed (5GB)

## Input

In [7]:
# Process email in json file
sender, reply_subject, email = input_email_processing("email.json")

NameError: name 'input_email_processing' is not defined

## Classification

import the classification model

In [5]:
model = load_model('email_classification_model.h5', custom_objects={'KerasLayer': hub.KerasLayer})

2024-04-07 10:08:44.696710: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [ ]:
new_email = reply_subject + "[SEP]" + email 
predictions = model.predict([new_email])
print(predictions)
isPhishing = False

## Reply Generation

In [ ]:
if isPhishing:
    template = generate_prompt_from_template(email)
    reply = generate_reply(template)

Llama.generate: prefix-match hit

llama_print_timings:        load time =   10281.91 ms
llama_print_timings:      sample time =       3.84 ms /    22 runs   (    0.17 ms per token,  5723.20 tokens per second)
llama_print_timings: prompt eval time =   16151.57 ms /   201 tokens (   80.36 ms per token,    12.44 tokens per second)
llama_print_timings:        eval time =    3292.38 ms /    21 runs   (  156.78 ms per token,     6.38 tokens per second)
llama_print_timings:       total time =   19510.33 ms /   222 tokens


## Output

In [ ]:
if isPhishing:
    output_reply_processing(sender, reply_subject, reply)
    print("Reply email created")
else:
    print("Email is not phishing")